In [1]:
import os
import json

os.chdir('..')
os.listdir('data/seeds')

['adversary-org.jsonl']

In [2]:
# Reading from jsonlines file
with open('data/seeds/adversary-org.jsonl', 'rb') as f:
    lines = f.readlines()
lines = [json.loads(line.decode('utf-8')) for line in lines]
line = lines[0]

import torch
from transformers import AdamW

from src.data import Data_Handler

handler = Data_Handler()
references = []
for line in lines:
    references.append( handler.process_prodigy_annot(line) )
    
unique_texts = list(set([reference[0].doc.text for reference in references]))
references

[[<src.data.Doc_Tokens at 0x1b5aa8b04c8>, [(0, 2), (76, 78)]],
 [<src.data.Doc_Tokens at 0x1b5aa8b0bc8>, [(2, 3), (5, 9), (31, 32)]],
  [(12, 13), (45, 46), (89, 90), (72, 74)]]]

### Prepare model & references

In [3]:
from src.dygie_ent import Dygie_Ent
model = Dygie_Ent()
model.references['adversary'] = references
model.get_prototypes();

### Prepare input text

In [4]:
text = "The activity of the advanced hacker group the researchers call Silence has increased significantly over the past year. Victims in the financial sector are scattered across more than 30 countries and financial losses have quintupled.\n The group started timidly in 2016, learning the ropes by following the path beaten by other hackers. Since then, it managed to steal at least $4.2 million, initially from banks in the former Soviet Union, then from victims in Europe, Latin America, Africa, and Asia.\n Researchers at Group-IB, Singapore-based cybersecurity company specializing in attack prevention, tracked Silence early on and judged its members to be familiar with white-hat security activity.\n A report last year\xa0details the roles of Silence hackers, their skills, failures, and successful bank heists"
text

'The activity of the advanced hacker group the researchers call Silence has increased significantly over the past year. Victims in the financial sector are scattered across more than 30 countries and financial losses have quintupled.\n The group started timidly in 2016, learning the ropes by following the path beaten by other hackers. Since then, it managed to steal at least $4.2 million, initially from banks in the former Soviet Union, then from victims in Europe, Latin America, Africa, and Asia.\n Researchers at Group-IB, Singapore-based cybersecurity company specializing in attack prevention, tracked Silence early on and judged its members to be familiar with white-hat security activity.\n A report last year\xa0details the roles of Silence hackers, their skills, failures, and successful bank heists'

In [5]:
doc = handler.process_sentence(text)
# for tok in doc.doc:
#     print(tok.i, tok)
ent_spans = [(10,10), (38,39), (111, 111), (138,139)]

### Test forward

In [6]:
probs = model(doc)
probs.shape

torch.Size([594, 2])

### Test eval function

In [7]:
model.evaluate(references)

([1.0], [0.0010660980810234541], [0.0021299254526091584])

### Test update

In [ ]:
model.update(doc, ent_spans)

### Test accept references

In [ ]:
model.accept_reference(doc, 'adversary', ent_spans)

In [ ]:
model.references

### Test save and load model

In [ ]:
model.save('models/test_checkpoint')

In [ ]:
model.load('models/test_checkpoint')